## Create Sampled Dataset of A2D2


In [1]:
import argparse
import os
import yaml
import numpy as np
import numpy.linalg as la
import open3d as o3d
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from matplotlib import cm
from functools import partial
import concurrent.futures
import random

from sphere import Sphere
from dh_grid import DHGrid
from laserscan import SemLaserScan

import os
import time
import matplotlib.pyplot as plt
import numpy as np
import json
import pprint
from os.path import join
import glob
import cv2
import open3d as o3d


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
dataroot = '/media/berlukas/Data/jd/camera lidar semantic'
sequences = f'{dataroot}/sequences'
export_ds = f'{dataroot}/processed'
segments = os.listdir(sequences)
sensors = ['cam_front_center', 'cam_front_left', 'cam_front_right', 'cam_rear_center', 'cam_side_left', 'cam_side_right']
# sensors = ['cam_front_center']

with open (f'{dataroot}/cams_lidars.json', 'r') as f:
    config = json.load(f)

print(f'Found {len(segments)} elements in the folder.')

Found 1 elements in the folder.


In [3]:
for segment in segments:
    root_path = f'{sequences}/{segment}/lidar'
    lidar_files = {}
    for lidar in sensors:
        sensor_path = f'{root_path}/{lidar}/*.npz'                
        lidar_files[lidar] = sorted(glob.glob(sensor_path))

In [ ]:
def undistort_image(image, cam_name):
    if cam_name in ['front_left', 'front_center', \
                    'front_right', 'side_left', \
                    'side_right', 'rear_center']:
        # get parameters from config file
        intr_mat_undist = \
                  np.asarray(config['cameras'][cam_name]['CamMatrix'])
        intr_mat_dist = \
                  np.asarray(config['cameras'][cam_name]['CamMatrixOriginal'])
        dist_parms = \
                  np.asarray(config['cameras'][cam_name]['Distortion'])
        lens = config['cameras'][cam_name]['Lens']
        
        if (lens == 'Fisheye'):
            return cv2.fisheye.undistortImage(image, intr_mat_dist,\
                                      D=dist_parms, Knew=intr_mat_undist)
        elif (lens == 'Telecam'):
            return cv2.undistort(image, intr_mat_dist, \
                      distCoeffs=dist_parms, newCameraMatrix=intr_mat_undist)
        else:
            return image
    else:
        return image

def extract_semantic_file_name_from_image_file_name(file_name_image):
    file_name_semantic_label = file_name_image.split('/')
    file_name_semantic_label = file_name_semantic_label[-1].split('.')[0]    
    file_name_semantic_label = file_name_semantic_label.split('_')
    file_name_semantic_label = file_name_semantic_label[0] + '_' + \
                  'label_' + \
                  file_name_semantic_label[2] + '_' + \
                  file_name_semantic_label[3] + '.png'    
    return file_name_semantic_label

def create_open3d_pc(lidar, cam_image=None):
    # create open3d point cloud
    pcd = o3d.geometry.PointCloud()
    
    # assign point coordinates
    pcd.points = o3d.utility.Vector3dVector(lidar['points'])
    
    # assign colours
    if cam_image is None:
        median_reflectance = np.median(lidar['reflectance'])
        colours = colours_from_reflectances(lidar['reflectance']) / (median_reflectance * 5)
        
        # clip colours for visualisation on a white background
        colours = np.clip(colours, 0, 0.75)
    else:
        rows = (lidar['row'] + 0.5).astype(np.int)
        cols = (lidar['col'] + 0.5).astype(np.int)
        colours = cam_image[rows, cols, :] / 255.0
        
    pcd.colors = o3d.utility.Vector3dVector(colours)    
    return pcd

def get_pointcloud_at(lidar, semantic_image):           
    # xyz and intensity
    points = lidar['points']
    intensity = lidar['reflectance']
    
    # semantic labels from the image
    rows = (lidar['row'] + 0.5).astype(int)
    cols = (lidar['col'] + 0.5).astype(int)
    labels = semantic_image[rows, cols, :] / 255.0
    
    pc = np.column_stack((points, intensity, labels))
    return pc

n_sensors = len(sensors)
all_sem_clouds = []
for sensor in sensors:
    print(f'Processing files for sensor: {sensor}')
    for file_name_lidar in tqdm(lidar_files[sensor]):
        lidar = np.load(file_name_lidar)
        
        seq_name = file_name_lidar.split('/')[-4]        
        file_name_semantic_label = extract_semantic_file_name_from_image_file_name(file_name_lidar)
        file_name_semantic_label = join(sequences, seq_name, f'label/{sensor}/', file_name_semantic_label)        
        semantic_image = cv2.imread(file_name_semantic_label)                
        semantic_image_undistorted = undistort_image(semantic_image, sensor[4:])
        pc = get_pointcloud_at(lidar, semantic_image_undistorted)
        all_sem_clouds.append(pc)
        
print(f'Found {len(all_sem_clouds)} pointclouds')

Processing files for sensor: cam_front_center


  0%|          | 0/975 [00:00<?, ?it/s]

In [14]:
def progresser(sample_idx, grid, auto_position=True, write_safe=False, blocking=True, progress=False):
    sample = all_sem_clouds[sample_idx]
    sample_sphere = Sphere(sample)
    features = sample_sphere.sampleUsingGrid(grid)
    return features

bw = 50
grid, _ = DHGrid.CreateGrid(bw)
print(f"Loading complete. Computing features...")
# parallel
sem_idx = np.arange(0, len(all_sem_clouds))
sample_func = partial(progresser, grid=grid)
sem_features = process_map(sample_func, sem_idx, max_workers=16)            

filename = f"{export_ds}/clouds.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}.")

filename = f'archive'
np.save(f'{export_ds}/{filename}.npy', all_sem_clouds)
print(f'Saved clouds to {export_ds}/{filename}.npy')

Loading complete. Computing features...


  0%|          | 0/975 [00:00<?, ?it/s]

Wrote features to /media/berlukas/Data/jd/camera lidar semantic/processed/clouds.npy.


/usr/lib/python3/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Saved clouds to /media/berlukas/Data/jd/camera lidar semantic/processed/archive.npy


## Visualize extracted pointclouds

In [7]:
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def visualizeRawPointcloud(pcl, val):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl[:, 0:3])
    colors = mapIntensityToRGB(val)
#     colors = scan.sem_color_lut[pcl[:,4].astype(np.int)]
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.visualization.draw_geometries([pcd])    
    
def createGrid_old(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([n_grid * n_grid, 2])
    for i in range(n_grid):
        for j in range(n_grid):
            points[k, 0] = (np.pi*(2*i+1))/(4*bw)
            points[k, 1] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])
    return cart_grid

def create_sampling_sphere(bw):
    grid = createGrid_old(bw)
    xyz_grid = convertGridToEuclidean_old(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid.T

In [20]:
pc = all_sem_clouds[490]
visualizeRawPointcloud(pc, pc[:, 4])

In [23]:
cur_sem_cloud = sem_features[144]

cur_sem_cloud = np.reshape(cur_sem_cloud, (3, -1)).T
pc = create_sampling_sphere(bw)
points_xyz = pc.T[:,0:3]
points_xyzl = np.column_stack((points_xyz, cur_sem_cloud[:,2]))

visualizeRawPointcloud(points_xyzl, points_xyzl[:, 3])